<p style="text-align:center">
    <a href="https://nbviewer.jupyter.org/github/twMr7/Python-Machine-Learning/blob/master/09-Other_Utilities.ipynb">
        Open In Jupyter nbviewer
        <img style="float: center;" src="https://nbviewer.jupyter.org/static/img/nav_logo.svg" width="120" />
    </a>
</p>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/twMr7/Python-Machine-Learning/blob/master/09-Other_Utilities.ipynb)

# 9. 其他實用工具 Other Utilities

Python 的通用性來自於豐富的標準函式庫，本章介紹以下幾種常用的工具模組。
+ [**9.1 日期與時間（Date and Time）**](#module-datetime)
+ [**9.2 物件序列化（Python Object Serialization）**](#module-pickle)
+ [**9.3 亂數（Random Numbers）**](#module-random)
+ [**9.4 數學函數（Math Functions）**](#module-math)
+ [**9.5 檔案系統路徑（File System Paths）**](#module-pathlib)
+ [**9.6 資料型別提示（Type Hints）**](#type-hints)


<a id="module-datetime"></a>

## 9.1 日期與時間 Date and Time

Python 標準函式庫中的 [`datetime`](https://docs.python.org/3/library/datetime.html#datetime-objects) 模組可以用來處理日期時間相關的資料，包含了 `date`, `time`, `datetime`, `timedelta`, `timezone` 等型別。

In [1]:
# 載入 datetime 模組
from datetime import datetime

In [2]:
# 現在的日期時間，返回 datetime 型別
t1 = datetime.now()
t1

datetime.datetime(2021, 3, 9, 16, 4, 26, 77914)

In [3]:
# 轉成字串 str 型別
str(t1)

'2021-03-09 16:04:26.077914'

In [4]:
# 現在的日期時間，轉成指定格式的字串
t1.strftime('%m/%d/%Y %H:%M:%S')

'03/09/2021 16:04:26'

In [5]:
# 從日期時間的字串轉成 datetime 型別
t2 = datetime.strptime('2020-10-28  15:10:00', '%Y-%m-%d %H:%M:%S')

# 比較兩個 datetime
if (t1 > t2):
    print('t1 比 t2 晚', t1 - t2)
else:
    print('t2 比 t1 晚', t2 - t1)

t1 比 t2 晚 132 days, 0:54:26.077914


標準函式庫還有另外一個 `time` 模組，提供了專門用來處理時間相關的函式，大多是從系統的C函式庫來的比較低階的處理。

In [6]:
# time 模組也有 strftime 可以用來格式化時間字串
import time
time.strftime('%Y%m%d %H%M%S')

'20210309 160426'

<a id="module-pickle"></a>

## 9.2 物件序列化 Python Object Serialization

Python 標準函式庫中的 [`pickle`](https://docs.python.org/3/library/pickle.html#module-pickle) 模組，提供了將 Python 物件序列化（serializing）及解序列化（de-serializing）的方法。 序列化指的是將物件階層轉換成位元組串流（byte stream），以方便物件的儲存、網路傳送、以及不同平臺的互通交換，反向的解序列化操作則是將位元組串流轉換成物件階層。

+ `pickle` 模組可以將物件儲存至檔案，或從檔案載入物件，檔案的存取需使用 binary 模式。
+ `pickle` 模組提供的序列化功能只適用於 Python 物件專用，標準函式庫中另外有跨平臺及程式語言的通用型的序列化模組 [`json`](https://docs.python.org/3/library/json.html#module-json)，但 `json` 只支援較少的 Python 內建物件型別。

In [7]:
# 載入 pickle 模組
import pickle

In [8]:
# 建立一個數據記錄的結構
tformat = '%Y-%m-%d %H:%M:%S'
record = [
    {'時間':datetime.strptime('2019-04-03 10:35:58', tformat), '體溫':37.0, '速度':35.0, '心率':92},
    {'時間':datetime.strptime('2019-04-03 10:37:00', tformat), '體溫':37.1, '速度':33.8, '心率':97},
    {'時間':datetime.strptime('2019-04-03 10:37:59', tformat), '體溫':37.4, '速度':35.5, '心率':99}
]

In [9]:
# 開啟新的 binary 檔案，用 pickle 將 record 物件 serialize
pfile = open('record.pkl', 'wb')
pickle.dump(record, pfile)
pfile.close()

In [10]:
# 讀入檔案，將 record 物件 de-serialize
pfile = open('record.pkl', 'rb')
record2 = pickle.load(pfile)
pfile.close()
record2

[{'時間': datetime.datetime(2019, 4, 3, 10, 35, 58),
  '體溫': 37.0,
  '速度': 35.0,
  '心率': 92},
 {'時間': datetime.datetime(2019, 4, 3, 10, 37),
  '體溫': 37.1,
  '速度': 33.8,
  '心率': 97},
 {'時間': datetime.datetime(2019, 4, 3, 10, 37, 59),
  '體溫': 37.4,
  '速度': 35.5,
  '心率': 99}]

<a id="module-random"></a>

## 9.3 亂數 Random Numbers

Python 標準函式庫中的 [`random`](https://docs.python.org/3/library/random.html) 模組，提供了擬隨機（pseudo-random）亂數產生的方法。

+ `random()` - 返回下一個 [0.0, 1.0) 區間內的隨機實數。
+ `randrange(start, stop[, step])` - 返回下一個 [start, stop) 區間內的隨機整數。
+ `randint(a, b)` - 返回下一個 [a, b] 區間內的隨機整數，同 `randrange(a, b+1)`。
+ `choice(seq)` - 從 seq 序列中隨機選取其中一個成員。
+ `shuffle(seq)` - 將 seq 序列中的元素順序重新隨機排列，序列必須是可就地變更的容器類別。
+ `sample(seq, k)` - 從 seq 序列或集合中，返回隨機選取 k 個樣本的 List 清單。

In [11]:
# 載入 random 模組
import random

In [12]:
# 產生 100 個隨機實數數列
Lr = [random.random() for x in range(100)]
print(Lr)

[0.40154229062738966, 0.2205187482027775, 0.3995984846410303, 0.7800394117557153, 0.9983850382849827, 0.6955912892247579, 0.1379734873046149, 0.3475459344373685, 0.511606908763249, 0.9604733519968859, 0.6258737406943511, 0.8803731190567751, 0.41784888955668675, 0.041731465855160854, 0.24283266731620556, 0.8605788200434756, 0.5838071424955392, 0.8943948588894899, 0.6879191220621628, 0.4399734507263646, 0.43952504493284417, 0.2815103066448451, 0.5825533392285963, 0.1814309528459701, 0.15733397532743654, 0.29031334280823395, 0.044509203008678666, 0.9397167858806349, 0.9051512607415323, 0.23055602942667075, 0.6935453734982807, 0.11389171117889751, 0.7456723402260754, 0.4360312396543967, 0.09141913767212051, 0.8328101663262963, 0.992076994270165, 0.9307157788888251, 0.1194554075808889, 0.5449471629056049, 0.5031763724157997, 0.9001236072204817, 0.872656358914501, 0.34657183313937157, 0.7431189766909563, 0.47983759658709024, 0.05148877252160444, 0.16786059354345417, 0.12163206066131893, 0.80

In [13]:
# 產生 100 個隨機整數數列
Li = [random.randint(1, 100) for x in range(100)]
print(Li)

[22, 60, 75, 15, 27, 2, 54, 70, 82, 84, 68, 62, 76, 16, 44, 77, 67, 82, 41, 26, 4, 17, 99, 91, 48, 35, 9, 2, 19, 27, 14, 22, 13, 52, 51, 73, 87, 70, 68, 2, 92, 15, 32, 44, 81, 41, 73, 55, 84, 63, 35, 66, 44, 38, 78, 41, 75, 94, 60, 76, 73, 10, 30, 17, 75, 90, 34, 13, 47, 23, 81, 46, 4, 59, 14, 72, 2, 100, 41, 74, 22, 87, 83, 88, 100, 32, 10, 58, 97, 9, 29, 67, 40, 96, 44, 96, 83, 87, 41, 29]


In [14]:
# 從數列中隨機選取 10 個樣本，產生新的隨機數列
[x * y for x, y in zip(random.sample(Lr, 10), random.sample(Li, 10))]

[6.856435325409038,
 35.77891071549454,
 34.27766288713768,
 35.752381695302766,
 24.345695122866527,
 1.5464070929769358,
 67.94225185888423,
 5.805790491071043,
 22.797900143048345,
 16.521549335729148]

<a id="module-math"></a>

## 9.4 數學函數 Math Functions

Python 標準函式庫中的 [`math`](https://docs.python.org/3/library/math.html) 模組，提供了用於實數運算的常用函數。

In [15]:
# 載入 math 模組
import math

In [16]:
# 內建函式的 sum() 在浮點數運算的精度不足
print(sum([.1, .1, .1, .1, .1, .1, .1, .1, .1, .1]))

0.9999999999999999


In [17]:
# math 模組的 fsum() 可避免精度的誤差
print(math.fsum([.1, .1, .1, .1, .1, .1, .1, .1, .1, .1]))

1.0


In [18]:
# cosine 180 度
print('cosine(pi) =', math.cos(math.pi))

cosine(pi) = -1.0


In [19]:
# sine 90 度
print('sine(pi/2) =', math.sin(math.radians(90)))

sine(pi/2) = 1.0


<a id="module-pathlib"></a>

## 9.5 檔案系統路徑 File System Paths

Python 標準函式庫中的 [`pathlib`](https://docs.python.org/3/library/pathlib.html) 模組，提供了通用於不同平台的檔案系統路徑操作，`Path` 物件可以比較、解析路徑的組成部份、也可以串接重組，主要有以下屬性：

+ `Path.drive` - 目標路徑的磁碟代號
+ `Path.root` - 目標路徑的根目錄
+ `Path.parent` - 目標路徑的上層目錄
+ `Path.name` - 目標路徑最後部份的名字
+ `Path.suffix` - 目標路徑最後部份的副檔名
+ `Path.stem` - 目標路徑最後部份去除副檔名的名字

常用的 `Path` 類別方法如下：

+ `Path.cwd()` - 目前工作目錄。
+ `Path.home()` - 登入使用者的家目錄。
+ `Path(str)` - 從字串 str 建立路徑物件。
+ `Path.exists()` - 路徑的檔案或目錄是否存在。
+ `Path.glob(pattern)` - 返回生成函式，用來列出路徑下符合指定 pattern 的所有檔案或目錄。
+ `Path.is_dir()` - 檢查路徑的目標是否爲目錄。
+ `Path.is_file()` - 檢查路徑的目標是否爲檔案。
+ `Path.iterdir()` - 當目標路徑爲目錄時，用來迭代尋訪目錄下的所有檔案。
+ `Path.mkdir()` - 當目標路徑爲目錄時，爲該目標建立目錄。
+ `Path.rename(new_name)` - 重新命名檔案。
+ `Path.open(mode)` - 功能同內建函式 `open()`，使用指定模式開啓檔案，返回檔案物件。 


In [20]:
# 載入 Path 類別
from pathlib import Path

In [21]:
# 列出目前工作目錄下所有的檔案及目錄
pwd = Path.cwd()
print('Current working directory: ', pwd)
for f in pwd.iterdir():
    print(f.name)

Current working directory:  /media/james/DATA/Users/James/GoogleDrive/Code/Lecture/Python-Machine-Learning/Lecture-Notes
.git
.gitignore
.ipynb_checkpoints
01-Getting_Started.ipynb
02-Syntax_Overview_1.ipynb
03-Syntax_Overview_2.ipynb
04-String_Operations.ipynb
05-List_Operations.ipynb
06-Tuple_Operations.ipynb
07-Dict_Operations.ipynb
08-File_Operations.ipynb
09-Other_Utilities.ipynb
10-Coding_Project.ipynb
11-Numpy_Vectorized_Computation.ipynb
12-Matplotlib_Data_Visualization.ipynb
13-Pandas_Data_Processing.ipynb
14-Sklearn_Building_A_Machine_Learning_Model.ipynb
15-Sklearn_Data_Preprocessing.ipynb
16-Sklearn_Best_Practice_Techniques.ipynb
17-Artificial_Neural_Network_with_tf_Keras.ipynb
18-ANN_Case_Studies.ipynb
19-Practical_Autoencoders.ipynb
20-CNN_Fundamental.ipynb
dataset
README.md
record.pkl


In [22]:
# 建立一個記錄檔案名字與副檔名對照的字典
{f.stem:f.suffix for f in pwd.iterdir() if f.is_file()}

{'.gitignore': '',
 '01-Getting_Started': '.ipynb',
 '02-Syntax_Overview_1': '.ipynb',
 '03-Syntax_Overview_2': '.ipynb',
 '04-String_Operations': '.ipynb',
 '05-List_Operations': '.ipynb',
 '06-Tuple_Operations': '.ipynb',
 '07-Dict_Operations': '.ipynb',
 '08-File_Operations': '.ipynb',
 '09-Other_Utilities': '.ipynb',
 '10-Coding_Project': '.ipynb',
 '11-Numpy_Vectorized_Computation': '.ipynb',
 '12-Matplotlib_Data_Visualization': '.ipynb',
 '13-Pandas_Data_Processing': '.ipynb',
 '14-Sklearn_Building_A_Machine_Learning_Model': '.ipynb',
 '15-Sklearn_Data_Preprocessing': '.ipynb',
 '16-Sklearn_Best_Practice_Techniques': '.ipynb',
 '17-Artificial_Neural_Network_with_tf_Keras': '.ipynb',
 '18-ANN_Case_Studies': '.ipynb',
 '19-Practical_Autoencoders': '.ipynb',
 '20-CNN_Fundamental': '.ipynb',
 'README': '.md',
 'record': '.pkl'}

In [23]:
# 列出目前工作目錄下所有副檔名是 .ipynb 的檔案
print([f.name for f in pwd.glob('*.ipynb')])

['01-Getting_Started.ipynb', '02-Syntax_Overview_1.ipynb', '03-Syntax_Overview_2.ipynb', '04-String_Operations.ipynb', '05-List_Operations.ipynb', '06-Tuple_Operations.ipynb', '07-Dict_Operations.ipynb', '08-File_Operations.ipynb', '09-Other_Utilities.ipynb', '10-Coding_Project.ipynb', '11-Numpy_Vectorized_Computation.ipynb', '12-Matplotlib_Data_Visualization.ipynb', '13-Pandas_Data_Processing.ipynb', '14-Sklearn_Building_A_Machine_Learning_Model.ipynb', '15-Sklearn_Data_Preprocessing.ipynb', '16-Sklearn_Best_Practice_Techniques.ipynb', '17-Artificial_Neural_Network_with_tf_Keras.ipynb', '18-ANN_Case_Studies.ipynb', '19-Practical_Autoencoders.ipynb', '20-CNN_Fundamental.ipynb']


<a id="type-hints"></a>

## 9.6 資料型別提示 Type Hints

版本 3.5 之後的 Python 在執行時期都有支援資料型別提示的語法，不需要載入特別的模組。

```
def function(arg: arg_type) -> return_type:
    statements
    return value
```

Python 是動態型別的程式語言，沒有強制變數或函式參數要事先宣告型別，但在大型的專案中，有型別的提示可以讓程式的結構設計具備較高的可讀性。 進階的型別支援功能可以透過載入 [`typing`](https://docs.python.org/3/library/typing.html) 模組來取得。


In [25]:
# def 函式名稱(參數: 型別) -> 回傳型別
def jiume(who: str) -> str:
    return who + ' >.^ '

jiume('Mary')

'Mary >.^ '

In [26]:
def addmyself(myself: int) -> int:
    return myself + myself

addmyself(5)

10

In [27]:
from typing import Any

def triple(what: Any) -> Any:
    return what * 3

print(triple(jiume('Mary')))
print(triple(addmyself(5)))

Mary >.^ Mary >.^ Mary >.^ 
30


### § 型別別名 Type Aliases

當某個型別定義在很深層的套件的模組裡時，使用別名可以讓程式看起來簡潔。

In [28]:
from typing import Tuple
from math import hypot

point3d = Tuple[float, float, float]

def distance3d(p1: point3d, p2: point3d) -> float:
    return hypot(*[(p1x - p2x) for p1x, p2x in zip(p1, p2)])


In [29]:
a = (1, 2, 3)
b = (3., 2., 1.)
print('distance between points', a, 'and', b, '=', distance3d(a, b))

distance between points (1, 2, 3) and (3.0, 2.0, 1.0) = 2.8284271247461903
